# Using Keras to Train a Model on Images and Masks

Unfortunately this is not working either.  
I don't believe the models are able to train on the images and the masks, but the masks need to be used to generate annotations for labels to train the model.

In [1]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()

# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# better progress bars for jupyter and won't kill model fit
# Command line the following line to get tqdm installed properly
# python -m pip install tqdm
from keras_tqdm import TQDMNotebookCallback
# keras, model definition...
cb = TQDMNotebookCallback()
setattr(cb,'on_train_batch_begin',lambda x,y:None)
setattr(cb,'on_train_batch_end',lambda x,y:None)

In [3]:
from __future__ import print_function
from keras.datasets import cifar10
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import time
import glob
import cv2
import matplotlib.pyplot as plt
import keras

#### Constants

In [64]:
batch_size = 32
num_classes = 10
epochs = 2
data_augmentation = False
num_predictions = 20
save_dir = os.path.join('../data/')
model_name = 'cnn_satellite_trained_model.h5'

### Read in Images and Masks

In [5]:
images = glob.glob('../images/training/build/0/*.tif')
img_list=[]
for item in images:
    
    item = cv2.imread(item)
    item = cv2.resize(item, (256, 256))
    item = img_to_array(item)
    img_list.append(item)

In [6]:
len(img_list)

180

In [7]:
img_list[0].shape

(256, 256, 3)

In [8]:
masks = glob.glob('../images/training/build_mask/0/*.tif')
mask_list = []
for item in masks:
    
    item = cv2.imread(item)
    item = cv2.resize(item, (256, 256))
    item = img_to_array(item)
    mask_list.append(item)

In [9]:
len(mask_list)

180

In [10]:
mask_list[0].shape

(256, 256, 3)

In [11]:
mask_list

[array([[[255., 255., 255.],
         [255., 255., 255.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [255., 255., 255.],
         [  0.,   0.,   0.],
         [255., 255., 255.]],
 
        ...,
 
        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0.,   0.],
 

### Broken Section
Reshaping is causing input and target errors on model fit

In [12]:
data = np.array(img_list, dtype='float')

In [13]:
data

array([[[[ 62.,  75.,  73.],
         [176., 194., 197.],
         [ 60.,  95.,  78.],
         ...,
         [155., 180., 191.],
         [149., 165., 165.],
         [ 90., 118., 110.]],

        [[ 83., 102.,  97.],
         [160., 175., 181.],
         [ 43.,  72.,  56.],
         ...,
         [120., 134., 135.],
         [ 57.,  63.,  59.],
         [ 58.,  66.,  67.]],

        [[ 68.,  74.,  77.],
         [ 93., 112., 110.],
         [ 85., 106., 106.],
         ...,
         [221., 226., 222.],
         [ 59.,  59.,  55.],
         [ 83.,  89.,  85.]],

        ...,

        [[140., 159., 162.],
         [142., 160., 164.],
         [136., 154., 158.],
         ...,
         [105., 117., 117.],
         [ 98., 110., 111.],
         [107., 118., 120.]],

        [[ 24.,  35.,  33.],
         [ 70.,  88.,  87.],
         [ 50.,  50.,  50.],
         ...,
         [107., 117., 118.],
         [ 47.,  48.,  49.],
         [ 57.,  54.,  50.]],

        [[ 74.,  97., 106.],
       

In [14]:
labels = np.array(mask_list)

In [15]:
labels

array([[[[255., 255., 255.],
         [255., 255., 255.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [255., 255., 255.],
         [  0.,   0.,   0.],
         [255., 255., 255.]],

        ...,

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
       

In [88]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=.2, random_state=77)

In [89]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [90]:
y_train /= 255
y_test /= 255

### Keras Docs Code

In [91]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (144, 256, 256, 3)
144 train samples
36 test samples


In [92]:
print('y_train shape:', y_train.shape)
print(y_train.shape[0], 'train samples')
print(y_test.shape[0], 'test samples')

y_train shape: (144, 256, 256, 3)
144 train samples
36 test samples


In [107]:
y_train = y_train.reshape(144, -1, 2)
y_test = y_test.reshape(144, -1, 2)
print(y_train.shape)
print(y_test.shape)

(144, 98304, 2)
(144, 24576, 2)


In [108]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [109]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=12)

Using real-time data augmentation.


ValueError: Error when checking target: expected activation_66 to have 2 dimensions, but got array with shape (144, 24576, 2)

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)